## Workbook of tonal harmony
#### Analysis of the string quartet Op. 59 n.3 mov. 1 (Introduzione) from music21

In [1]:
from pcsPy import *
import pickle, copy
from IPython.display import Image 
sys.path.append('../')
from tonalHarmonyDefs import enharmonicDictionary, shortHands, scoreAnalysis, tonalPartition, showAnalysis

<IPython.core.display.Javascript object>

Here we define the tonal space using the ABC database. A tonal space is the matrix of all the operators that connect the chords of a chord library as roman numerals.
Typical models are built using combinations of basic chord symbols and possible extension. 

In [37]:
scoretvs = '/Users/marco/Dropbox (Personal)/Musica/Applications/musicntwrk/RESULTS/MUSICOLOGY/Beethoven/ABC-1.0/data/tsv/op. 127 No. 12/op127_no12_mov1.tsv'

In [38]:
abc_dataset = pd.read_csv(scoretvs,sep='\t')

In [10]:
abc_dataset

,chord,altchord,measure,beat,totbeat,timesig,op,no,mov,length,global_key,local_key,pedal,numeral,form,figbass,changes,relativeroot,phraseend
0,.Eb.I,NaN,1,1.0,1.0,2/4,127,12,1,2.5,Eb,I,NaN,I,NaN,NaN,NaN,NaN,False
1,V43,NaN,2,1.5,3.5,2/4,127,12,1,1.5,Eb,I,NaN,V,NaN,43.0,NaN,NaN,False
2,I,NaN,3,1.0,5.0,2/4,127,12,1,2.5,Eb,I,NaN,I,NaN,NaN,NaN,NaN,False
3,V2,NaN,4,1.5,7.5,2/4,127,12,1,1.5,Eb,I,NaN,V,NaN,2.0,NaN,NaN,False
4,I6,NaN,5,1.0,9.0,2/4,127,12,1,1.0,Eb,I,NaN,I,NaN,6.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,I6(72),NaN,280,1.0,822.0,3/4,127,12,1,1.0,Eb,I,NaN,I,NaN,6.0,72,NaN,False
554,I6(7),NaN,280,2.0,823.0,3/4,127,12,1,1.0,Eb,I,NaN,I,NaN,6.0,7,NaN,False
555,I64,NaN,280,3.0,824.0,3/4,127,12,1,1.0,Eb,I,NaN,I,NaN,64.0,NaN,NaN,False
556,V7,NaN,281,1.0,825.0,3/4,127,12,1,3.0,Eb,I,NaN,V,NaN,7.0,NaN,NaN,False


In [34]:
rn = []
for n in range(len(abc_dataset)):
    if str(abc_dataset['form'][n]) != 'nan':
        roman = str(abc_dataset['numeral'][n])
        extension = str(abc_dataset['form'][n])
        if extension == '%':
            extension = '/o'
        if extension == 'M':
            extension = '7'
        if extension == 'Ger':
            extension = '7'
        rn.append(roman+extension)
    else:
        roman = str(abc_dataset['numeral'][n])
        if roman == 'nan':
            pass
        else:
            rn.append(roman)
rn = Remove(rn)

In [35]:
sorted(rn)

['#iv/o',
 '#vi',
 '#viio',
 'Ger',
 'I',
 'I+',
 'II',
 'III',
 'IV',
 'IV+',
 'IV7',
 'V',
 'VI',
 'VII',
 'i',
 'ii',
 'ii/o',
 'iii',
 'iio',
 'iv',
 'ivo',
 'v',
 'vi',
 'vii/o',
 'viio']

In [24]:
str(abc_dataset['form'][1])

'nan'

In [2]:
score = '/Users/marco/Dropbox (Personal)/Musica/Applications/musicntwrk/RESULTS/MUSICOLOGY/Beethoven/ABC-1.0/data/mscx/op. 127 No. 12/op127_no12_mov1.musicxml'

In [3]:
# score = 'Op59n3m1_Intro.musicxml'
seq,chords = readScore(score,music21=False,show=False)

In [ ]:
# To write chords and corresponding operators - uses musescore
WRITEscoreOps(seq,w=True)

In [4]:
dnodes,dedges,dcounts,deg,modul,Gx,Gxu = scoreNetwork(seq,ntx=True,general=True)
print('average degree = ',deg,' modularity = ',modul)

average degree =  3.247967479674797  modularity =  0.4332518980199036


In [98]:
# Write to csv for visualization in Gephi or other network drawing software (see figure below)
dnodes.to_csv('nodesOp127n12m1.csv')
dedges.to_csv('edgesOp127n12m1.csv')
# Get modularity scores
# part = cm.best_partition(Gxu) #,resolution=0.4)

## ![title](Op127n12m1.png)

In [5]:
moduldict,modul = tonalPartition(seq,chords,dnodes,Gx,Gxu,resolution=1.0,display=True)
# modul.to_csv('~/Desktop/modul.csv')

In [58]:
modul[['Label','Degree','Modularity']].query('Modularity == 7').sort_values(by=['Degree'],ascending=False)

,Label,Degree,Modularity


In [6]:
# Dictionary of modularity classes and relative keys
keydict = {0:'E-', 1:'E-', 2:'E-', 3:'E-', 4:'E-', 5:'E-', 6:'E-'}

In [26]:
# manual control of modulations
keychange = {4:'E-',11:'E-',27:'E-',28:'E-',29:'E-'}
# manual control of roman numerals
altrn = None #{2:'II'}
# manual elimination of "unknown" chords
seq[40] = seq[39]
seq[44] = seq[43]

In [27]:
nxt,rn,ops = scoreAnalysis(seq,moduldict,keydict,keychange=keychange,altrn=altrn,table='../tonal.harmony.minimal')

modulation at or before chord no.  57
check error


In [44]:
analysis = showAnalysis(nxt,chords,seq,rn,ops,keydict,moduldict,last=False,display=True)

In [43]:
def showAnalysis(nt,chords,seq,rn,ops,keydict,moduldict,wops=False,last=False,display=False):
# Create dictionary of score analysis
    reference = []
    for n in range(nt):
        try:
            chord = ''.join(m21.chord.Chord(PCSet(seq[n]).normalOrder().tolist()).pitchNames)
            entry = [PCSet(seq[n]).normalOrder(),chord,rn[n],ops[n],keydict[moduldict[chord]],moduldict[chord]]
            reference.append(entry)
        except:
            pass
    if last:
    # Add last chord
        ops.append(' ')
        chord = ''.join(m21.chord.Chord(PCSet(seq[nt]).normalOrder().tolist()).pitchNames)
        entry = [PCSet(seq[nt]).normalOrder(),chord,rn[nt],ops[nt],keydict[moduldict[chord]]]
        reference.append(entry)

    # Set dictionary as pandas dataframe
    analysis = pd.DataFrame(reference,columns=['pcs','chord','rn','ops','key','modul'])
    
    if display:
        # display the analyzed score
        l = 0
        analyzed = copy.deepcopy(chords)
        for c in analyzed.recurse().getElementsByClass('Chord'):
            c.closedPosition(forceOctave=4,inPlace=True)
            c.addLyric('')
            c.addLyric('')
            if wops: c.addLyric(str(ops[l]))
            c.addLyric(str(rn[l]))
            if l < nt-1: 
                l += 1
            else: 
                break
        analyzed.show('musicxml')
    return(analysis)


In [32]:
for i,n in enumerate(rn):
    print(i,n,ops[i],seq[i],chords[i])

0 I O(0,0,0) [3, 7, 10] <music21.metadata.Metadata object at 0x1a1b782ed0>
1 I O(2,1,0,-1) [3, 7, 10] P1: Violin I: Violin I (2)
2 V7 O(0,0,0,0) [2, 5, 8, 10] <music21.stream.Measure 1 offset=0.0>
3 V7 O(1,-2,-1,0) [2, 5, 8, 10] <music21.stream.Measure 2 offset=2.0>
4 I O(0,0,0) [3, 7, 10] <music21.stream.Measure 3 offset=4.0>
5 I O(2,1,0,-1) [3, 7, 10] <music21.stream.Measure 4 offset=6.0>
6 V7 O(0,0,0,0) [2, 5, 8, 10] <music21.stream.Measure 5 offset=8.0>
7 V7 O(1,-2,-1,0) [2, 5, 8, 10] <music21.stream.Measure 6 offset=10.0>
8 I O(0,0,2) [3, 7, 10] <music21.stream.Measure 7 offset=12.0>
9 vi O(0,0,1) [0, 3, 7] <music21.stream.Measure 8 offset=15.0>
10 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 9 offset=18.0>
11 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 10 offset=21.0>
12 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 11 offset=24.0>
13 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 12 offset=27.0>
14 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 13 offset=30.0>
15 IV O(0,0,0) [8

In [14]:
nxt

40